# Use case
We want to fine-tune a model, which can generate SQL queries based on a natural language instruction, which can then be integrated into our BI tool. The goal is to reduce the time it takes to create a SQL query and make it easier for non-technical users to create SQL queries

In [1]:
import torch; assert torch.cuda.get_device_capability()[0] >= 8, 'Hardware not supported for Flash Attention'

In [2]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

HUGGINFACE_TOKEN  = os.environ['HUGGINFACE_TOKEN']

In [3]:
from huggingface_hub import login

login(
    token=HUGGINFACE_TOKEN,
    add_to_git_credential=True
)

Token is valid (permission: write).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /home/kris/.cache/huggingface/token
Login successful


# Dataset
It contains samples of natural language instructions, schema definitions and the corresponding SQL query.

In [4]:
from datasets import load_dataset

# Convert dataset to OAI messages
system_message = """You are an text to SQL query translator. Users will ask you questions in English and you will generate a SQL query based on the provided SCHEMA.
SCHEMA:
{schema}"""

def create_conversation(sample):
    return {
        "messages": [
            {"role": "system", "content": system_message.format(schema=sample["context"])},
            {"role": "user", "content": sample["question"]},
            {"role": "assistant", "content": sample["answer"]}
        ]
    }

# Load dataset from the hub
dataset = load_dataset("b-mc2/sql-create-context", split="train")
dataset = dataset.shuffle().select(range(12500))

In [5]:
dataset

Dataset({
    features: ['context', 'answer', 'question'],
    num_rows: 12500
})

In [6]:
dataset[0]

{'context': 'CREATE TABLE table_2554479_2 (tests_won_by_australia VARCHAR, series VARCHAR)',
 'answer': 'SELECT COUNT(tests_won_by_australia) FROM table_2554479_2 WHERE series = 10',
 'question': 'Name the total number of tests won by australia for series 10'}

In [7]:
# Convert dataset to OAI messages
dataset = dataset.map(create_conversation, remove_columns=dataset.features,batched=False)
# split dataset into 10,000 training samples and 2,500 test samples
dataset = dataset.train_test_split(test_size=2500/12500)

print(dataset["train"][0]["messages"])

# save datasets to disk
dataset["train"].to_json("train_dataset.json", orient="records")
dataset["test"].to_json("test_dataset.json", orient="records")

Map:   0%|          | 0/12500 [00:00<?, ? examples/s]

[{'content': 'You are an text to SQL query translator. Users will ask you questions in English and you will generate a SQL query based on the provided SCHEMA.\nSCHEMA:\nCREATE TABLE Drama_Workshop_Groups (Store_Name VARCHAR)', 'role': 'system'}, {'content': 'Show all the Store_Name of drama workshop groups.', 'role': 'user'}, {'content': 'SELECT Store_Name FROM Drama_Workshop_Groups', 'role': 'assistant'}]


Creating json from Arrow format:   0%|          | 0/10 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

1189376

In [8]:
from datasets import load_dataset

# Load jsonl data from disk
dataset = load_dataset("json", data_files="train_dataset.json", split="train")

Generating train split: 0 examples [00:00, ? examples/s]

# Model

In [9]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from trl import setup_chat_format

# Hugging Face model id
model_id = "tiiuae/falcon-rw-1b" 

# BitsAndBytesConfig int-4 config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, 
    bnb_4bit_use_double_quant=True, 
    bnb_4bit_quant_type="nf4", 
    bnb_4bit_compute_dtype=torch.bfloat16
)

# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    #attn_implementation="flash_attention_2",
    torch_dtype=torch.bfloat16,
    quantization_config=bnb_config
)
model.config.use_cache = False
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.padding_side = 'right' # to prevent warnings

# Sets the chat_template of the tokenizer, 
# which is used to format the input data into a chat-like format. 
# The default is chatml from OpenAI.
model, tokenizer = setup_chat_format(model, tokenizer)

2024-02-05 17:22:42.341852: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-05 17:22:42.436205: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/local/cuda-12.3/lib64:/usr/local/cuda-12.3/extras/CUPTI/lib64
2024-02-05 17:22:42.436216: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2024-02-05 17:22:42.933255: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dle

# Training the model

In [10]:
from peft import LoraConfig

# Tuning LoRA config for efficient fine-tuning of the model
peft_config = LoraConfig(
        lora_alpha=128,
        lora_dropout=0.05,
        r=256,
        bias="none",
        target_modules="all-linear",
        task_type="CAUSAL_LM",
)

In [11]:
from transformers import TrainingArguments

args = TrainingArguments(
    output_dir="model-text-to-sql", # directory to save and repository id
    num_train_epochs=3,                     # number of training epochs
    per_device_train_batch_size=3,          # batch size per device during training
    gradient_accumulation_steps=2,          # number of steps before performing a backward/update pass
    gradient_checkpointing=True,            # use gradient checkpointing to save memory
    optim="adamw_torch_fused",              # use fused adamw optimizer
    logging_steps=10,                       # log every 10 steps
    save_strategy="epoch",                  # save checkpoint every epoch
    learning_rate=2e-4,                     # learning rate, based on QLoRA paper
    bf16=True,                              # use bfloat16 precision
    tf32=True,                              # use tf32 precision
    max_grad_norm=0.3,                      # max gradient norm based on QLoRA paper
    warmup_ratio=0.03,                      # warmup ratio based on QLoRA paper
    lr_scheduler_type="constant",           # use constant learning rate scheduler
    push_to_hub=True,                       # push model to hub
    report_to="tensorboard",                # report metrics to tensorboard
)

In [12]:
from trl import SFTTrainer

max_seq_length = 1824 # max sequence length for model and packing of the dataset

trainer = SFTTrainer(
    model=model,
    args=args,
    train_dataset=dataset,
    peft_config=peft_config,
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    packing=True,
    dataset_kwargs={
        "add_special_tokens": False,  # We template with special tokens
        "append_concat_token": False, # No need to add additional separator token
    }
)

Generating train split: 0 examples [00:00, ? examples/s]

In [13]:
# start training, the model will be automatically saved to the hub and the output directory
trainer.train()

# save model
trainer.save_model()

  0%|          | 0/315 [00:00<?, ?it/s]

/home/kris/.local/lib/python3.10/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


{'loss': 1.4211, 'learning_rate': 0.0002, 'epoch': 0.1}
{'loss': 0.942, 'learning_rate': 0.0002, 'epoch': 0.19}
{'loss': 0.8478, 'learning_rate': 0.0002, 'epoch': 0.29}
{'loss': 0.7957, 'learning_rate': 0.0002, 'epoch': 0.38}
{'loss': 0.7793, 'learning_rate': 0.0002, 'epoch': 0.48}
{'loss': 0.7572, 'learning_rate': 0.0002, 'epoch': 0.57}
{'loss': 0.7289, 'learning_rate': 0.0002, 'epoch': 0.67}
{'loss': 0.7145, 'learning_rate': 0.0002, 'epoch': 0.76}
{'loss': 0.707, 'learning_rate': 0.0002, 'epoch': 0.86}
{'loss': 0.7182, 'learning_rate': 0.0002, 'epoch': 0.95}


/home/kris/.local/lib/python3.10/site-packages/peft/utils/save_and_load.py:160: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/home/kris/.local/lib/python3.10/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


{'loss': 0.6821, 'learning_rate': 0.0002, 'epoch': 1.05}
{'loss': 0.6553, 'learning_rate': 0.0002, 'epoch': 1.14}
{'loss': 0.6429, 'learning_rate': 0.0002, 'epoch': 1.24}
{'loss': 0.6478, 'learning_rate': 0.0002, 'epoch': 1.33}
{'loss': 0.6406, 'learning_rate': 0.0002, 'epoch': 1.43}
{'loss': 0.6395, 'learning_rate': 0.0002, 'epoch': 1.52}
{'loss': 0.6456, 'learning_rate': 0.0002, 'epoch': 1.62}
{'loss': 0.642, 'learning_rate': 0.0002, 'epoch': 1.71}
{'loss': 0.6446, 'learning_rate': 0.0002, 'epoch': 1.81}
{'loss': 0.6367, 'learning_rate': 0.0002, 'epoch': 1.9}
{'loss': 0.639, 'learning_rate': 0.0002, 'epoch': 2.0}


/home/kris/.local/lib/python3.10/site-packages/peft/utils/save_and_load.py:160: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/home/kris/.local/lib/python3.10/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


{'loss': 0.5759, 'learning_rate': 0.0002, 'epoch': 2.1}
{'loss': 0.5606, 'learning_rate': 0.0002, 'epoch': 2.19}
{'loss': 0.5627, 'learning_rate': 0.0002, 'epoch': 2.29}
{'loss': 0.5733, 'learning_rate': 0.0002, 'epoch': 2.38}
{'loss': 0.5727, 'learning_rate': 0.0002, 'epoch': 2.48}
{'loss': 0.5701, 'learning_rate': 0.0002, 'epoch': 2.57}
{'loss': 0.5636, 'learning_rate': 0.0002, 'epoch': 2.67}
{'loss': 0.5643, 'learning_rate': 0.0002, 'epoch': 2.76}
{'loss': 0.5783, 'learning_rate': 0.0002, 'epoch': 2.86}
{'loss': 0.5673, 'learning_rate': 0.0002, 'epoch': 2.95}


/home/kris/.local/lib/python3.10/site-packages/peft/utils/save_and_load.py:160: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


{'train_runtime': 1122.6105, 'train_samples_per_second': 1.681, 'train_steps_per_second': 0.281, 'train_loss': 0.6826381501697358, 'epoch': 3.0}


/home/kris/.local/lib/python3.10/site-packages/peft/utils/save_and_load.py:160: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


In [14]:
# free the memory again
del model
del trainer
torch.cuda.empty_cache()

In [15]:
import torch
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer, pipeline

peft_model_id = "./model-text-to-sql"
# peft_model_id = args.output_dir

# Load Model with PEFT adapter
model = AutoPeftModelForCausalLM.from_pretrained(
    peft_model_id,
    device_map="auto",
    torch_dtype=torch.float16
)
# load into pipeline
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM',

In [16]:
from datasets import load_dataset
from random import randint

# Load our test dataset
eval_dataset = load_dataset("json", data_files="test_dataset.json", split="train")
rand_idx = randint(0, len(eval_dataset))

# Test on sample
prompt = pipe.tokenizer.apply_chat_template(eval_dataset[rand_idx]["messages"][:2], tokenize=False, add_generation_prompt=True)
outputs = pipe(prompt, max_new_tokens=256, do_sample=False, temperature=0.1, top_k=50, top_p=0.1, eos_token_id=pipe.tokenizer.eos_token_id, pad_token_id=pipe.tokenizer.pad_token_id)

print(f"Query:\n{eval_dataset[rand_idx]['messages'][1]['content']}")
print(f"Original Answer:\n{eval_dataset[rand_idx]['messages'][2]['content']}")
print(f"Generated Answer:\n{outputs[0]['generated_text'][len(prompt):].strip()}")

Generating train split: 0 examples [00:00, ? examples/s]

/home/kris/.local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:392: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/kris/.local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:397: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Query:
Show the product type codes that have both products with price higher than 4500 and products with price lower than 3000.
Original Answer:
SELECT Product_Type_Code FROM Products WHERE Product_Price > 4500 INTERSECT SELECT Product_Type_Code FROM Products WHERE Product_Price < 3000
Generated Answer:
SELECT Product_Type_Code FROM Products WHERE Product_Price > 4500 AND Product_Price < 3000 AND Product_Type_Code = "laptop"


# Test predictions

In [17]:
from tqdm import tqdm

def evaluate(sample):
    prompt = pipe.tokenizer.apply_chat_template(sample["messages"][:2], tokenize=False, add_generation_prompt=True)
    outputs = pipe(prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95, eos_token_id=pipe.tokenizer.eos_token_id, pad_token_id=pipe.tokenizer.pad_token_id)
    predicted_answer = outputs[0]['generated_text'][len(prompt):].strip()
    if predicted_answer == sample["messages"][2]["content"]:
        return 1
    else:
        return 0

success_rate = []
number_of_eval_samples = 1000
# iterate over eval dataset and predict
for s in tqdm(eval_dataset.shuffle().select(range(number_of_eval_samples))):
    success_rate.append(evaluate(s))

# compute accuracy
accuracy = sum(success_rate)/len(success_rate)

print(f"Accuracy: {accuracy*100:.2f}%")

  1%|          | 9/1000 [00:15<28:32,  1.73s/it]/home/kris/.local/lib/python3.10/site-packages/transformers/pipelines/base.py:1123: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
100%|██████████| 1000/1000 [32:56<00:00,  1.98s/it]

Accuracy: 18.90%
